 Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas

https://hh.ru/search/vacancy?area=1&st=searchVacancy&text=%D0%A1%D0%B2%D0%B0%D1%80%D1%89%D0%B8%D0%BA&from=suggest_post&showClusters=false

<a class="bloko-button HH-Pager-Control" data-page="11" data-qa="pager-page" rel="nofollow" href="/search/vacancy?L_is_autosearch=false&amp;area=1&amp;clusters=true&amp;enable_snippets=true&amp;text=%D0%A1%D0%B2%D0%B0%D1%80%D1%89%D0%B8%D0%BA&amp;page=11">12</a>

https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&order_by=publication_time&text=%D0%A1%D0%B2%D0%B0%D1%80%D1%89%D0%B8%D0%BA&salary=
https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&order_by=publication_time&text=%D0%A1%D0%B2%D0%B0%D1%80%D1%89%D0%B8%D0%BA&page=11

In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests

In [2]:
search = input('Введите название вакансии')
main_link = f'https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&order_by=publication_time&text={search}&salary='
print(main_link)

Введите название вакансииПрограммист
https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&order_by=publication_time&text=Программист&salary=


In [3]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
response = requests.get(main_link, headers = header).text
html = bs(response,'lxml')

In [4]:
pager = html.findAll('span',{'class':'pager-item-not-in-short-range'})
last_page = pager[-1].find('a',{'class':'bloko-button HH-Pager-Control', 'data-qa': 'pager-page'})
link_last = last_page['href']
link_last

'/search/vacancy?area=1&clusters=true&enable_snippets=true&order_by=publication_time&search_field=name&text=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82&page=99'

In [5]:
global_name = 'https://hh.ru'
main_link_last = f'{global_name}{link_last}'

In [6]:
response_last = requests.get(main_link_last, headers = header).text
html_last = bs(response_last,'lxml')

In [7]:
names = html_last.findAll('div',{'class':'resume-search-item__name'})

In [8]:
import pandas as pd
result = pd.DataFrame()

In [9]:
names_list = []
for i in names:
    names_list.append(i.getText())

In [10]:
result['names'] = names_list

In [11]:
result

,names
0,Android developer
1,Веб-разработчик/PHP-программист
2,Программист-разработчик С/С++
3,Backend разработчик (golang)
4,Frontend-разработчик React
5,Java-разработчик
6,"Инженер-разработчик (Embedded, C++)"
7,Разработчик PHP
8,Senior web developer (backend)
9,Java разработчик


In [13]:
salary = []

In [14]:
salarys = html_last.findAll('div',{'class':'vacancy-serp-item__row vacancy-serp-item__row_header'})
salarys_all = []
for i in salarys:
    salarys_2 = i.find('div',{'class':'vacancy-serp-item__compensation'})
    salarys_all.append(salarys_2)
for i in range(len(salarys_all)-1):
    if salarys_all[i]:
        salarys_all[i] = salarys_all[i].getText()
salarys_all

[None,
 'до 150\xa0000 руб.',
 None,
 None,
 None,
 None,
 None,
 'от 120\xa0000 руб.',
 '160\xa0000-250\xa0000 руб.',
 None,
 None,
 None,
 None,
 None,
 None,
 'от 2\xa0800 USD',
 '100\xa0000-150\xa0000 руб.',
 'от 170\xa0000 руб.',
 '200\xa0000-287\xa0500 руб.',
 <div class="vacancy-serp-item__compensation" data-qa="vacancy-serp__vacancy-compensation">от 180 000 руб.</div>]

In [52]:
salarys_1 = []
salarys_2 = []

In [53]:
import re

In [56]:
for i in range(len(salarys_all)-1):
    if salarys_all[i]:
        hm = re.findall(r'^от_*', salarys_all[i])
        if hm:
            salarys_1.append(hm[0])
            salarys_2.append(hm[0])
    else:
        salarys_1.append('None')
        salarys_2.append('None')

In [57]:
salarys_1

['None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'от',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'от',
 'от']